In [ ]:
#################################### Formats Excel Spreadsheet ################################

In [1]:
import numpy as np
import pandas as pd

In [2]:
# loads the surveys as dataframes

df2016 = pd.read_excel('accountability2016schools.xlsx')
df2012 = pd.read_excel('accountability2012schools.xlsx')

In [3]:
dfstudentcount2016 = pd.read_excel('class_size2016.xlsx')
dfstudentcount2012 = pd.read_excel('class_size2012.xlsx')

# overrides school name of dfstudentounts withdf 201x
del dfstudentcount2012['school']
del dfstudentcount2016['school']
print(len(dfstudentcount2012), len(df2012))

1848 1828


In [4]:
# removes all rows where level is marked insufficient data

df2012 = df2012[df2012.level_2012 != 'Insufficient data']
df2012 = df2012[df2012.level_2012 != '']
print(len(df2012))

df2016 = df2016[df2016.level_2016 != 'Insufficient data']
df2016 = df2016[df2016.level_2016 != '']

# changes levelsl to ints

df2012['level_2012'] = df2012['level_2012'].apply(lambda x: [int(x) for x in x.split() if x.isdigit()][0])
df2016['level_2016'] = df2016['level_2016'].apply(lambda x: [int(x) for x in x.split() if x.isdigit()][0])

df2016 = df2016.merge(dfstudentcount2016, on='org_code')
df2016['student_count_2016'] = df2016['student_count']

df2012 = df2012.merge(dfstudentcount2012, on='org_code')
df2012['student_count_2012'] = df2012['student_count']

print(len(df2016))


1586
1620


In [5]:
df2012[:4]

,school,org_code,level_2012,school_type,student_count,student_count_2012
0,Abby Kelley Foster Charter Public (District) -...,4450105,1,Middle-High School or K-12,1426,1426
1,Abington - Abington High,10505,2,High School,498,498
2,Abington - Frolio Middle School,10405,2,Middle School,346,346
3,Abington - Woodsdale Elementary School,10015,1,Elementary School,315,315


In [6]:
df2016 = df2016.merge(dfstudentcount2016, on='org_code')
df2016['student_count_2016'] = df2016['student_count_x'] 
del df2016['student_count_y']
del df2016['student_count_x']

print(len(df2016))

df2012 = df2012.merge(dfstudentcount2012, on='org_code')
del df2012['student_count_y']
del df2012['student_count_x']

print(len(df2012))

df2016[:5]

1620
1572


,school,org_code,level_2016,school_type,student_count_2016
0,Abby Kelley Foster Charter Public (District) -...,4450105,2,Middle-High School or K-12,1420
1,Abington - Abington High,10505,1,High School,451
2,Abington - Beaver Brook Elementary School,10003,3,Elementary School,643
3,Abington - Frolio Middle School,10405,2,Middle School,342
4,Abington - Woodsdale Elementary School,10015,2,Elementary School,334


In [11]:
dfmerged = df2016.merge(df2012, on='org_code')
del dfmerged['school_y']
del dfmerged['school_type_y']

districtlst = []
# creates new row of just districts
def get_district(value):
    index = value.index(' -')
    if value[: index+1] not in districtlst:
        districtlst.append(value[: index+1])
         
    return value[: index+1]

dfmerged['district'] = dfmerged['school_x'].apply(lambda x : get_district(x))

# creates new row of just schools
def get_school(value): 
    index = value.index(' -') 
    return value[index + 3:]

dfmerged['school_x'] = dfmerged['school_x'].apply(lambda x: get_school(x))

dfmerged[:4]


,school_x,org_code,level_2016,school_type_x,student_count_2016,level_2012,student_count_2012,district
0,Abby Kelley Foster Charter Public School,4450105,2,Middle-High School or K-12,1420,1,1426,Abby Kelley Foster Charter Public (District)
1,Abington High,10505,1,High School,451,2,498,Abington
2,Frolio Middle School,10405,2,Middle School,342,2,346,Abington
3,Woodsdale Elementary School,10015,2,Elementary School,334,1,315,Abington


In [13]:
def weighted_score_2012():
    score_lst = []
    for index, row in dfmerged.iterrows():
        score = row['level_2012'] * row['student_count_2012']
        score_lst.append(score)
    return score_lst

dfmerged['score_2012'] = weighted_score_2012()

In [14]:
def weighted_score_2016():
    score_lst = []
    for index, row in dfmerged.iterrows():
        score = row['level_2016'] * row['student_count_2016']
        score_lst.append(score)
    return score_lst

dfmerged['score_2016'] = weighted_score_2016()

In [15]:
dfmerged

,school_x,org_code,level_2016,school_type_x,student_count_2016,level_2012,student_count_2012,district,score_2012,score_2016
0,Abby Kelley Foster Charter Public School,4450105,2,Middle-High School or K-12,1420,1,1426,Abby Kelley Foster Charter Public (District),1426,2840
1,Abington High,10505,1,High School,451,2,498,Abington,996,451
2,Frolio Middle School,10405,2,Middle School,342,2,346,Abington,692,684
3,Woodsdale Elementary School,10015,2,Elementary School,334,1,315,Abington,315,668
4,Academy Of the Pacific Rim Charter Public School,4120530,2,Middle-High School or K-12,525,1,493,Academy Of the Pacific Rim Charter Public (Dis...,493,1050
5,Acton-Boxborough Regional High,6000505,1,High School,1900,1,1989,Acton-Boxborough,1989,1900
6,Raymond J Grey Junior High,6000405,1,Middle School,895,2,939,Acton-Boxborough,1878,895
7,Acushnet Elementary School,30025,2,Elementary School,558,1,540,Acushnet,540,1116
8,Albert F Ford Middle School,30305,2,Middle School,426,2,463,Acushnet,926,852
9,Cheshire Elementary,6030004,2,Elementary School,248,2,179,Adams-Cheshire,358,496


In [16]:
dfdistrictlevelmerged = pd.DataFrame(districtlst, columns=['district'])

dfdistrictlevelmerged[:4]

,district
0,Abby Kelley Foster Charter Public (District)
1,Abington
2,Academy Of the Pacific Rim Charter Public (Dis...
3,Acton-Boxborough


In [17]:
def number_of_schools():
    lst_schools = []
    count = 1
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        #print(current_district)
        if row['district'] == current_district:
            count += 1
        else:
            current_district = row['district']
            lst_schools.append(count)
            count = 1
    return lst_schools

school_count_lst = number_of_schools()

school_count_lst.pop(0)

school_count_lst = [1] + school_count_lst

school_count_lst.append(2)
print(len(districtlst))

print(len(school_count_lst))

dfdistrictlevelmerged['school count'] = school_count_lst

369
369


In [18]:
dfdistrictlevelmerged[:4]

,district,school count
0,Abby Kelley Foster Charter Public (District),1
1,Abington,3
2,Academy Of the Pacific Rim Charter Public (Dis...,1
3,Acton-Boxborough,2


In [21]:
def district_score_2012():
    score_lst = []
    score = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            score += row['score_2012']
        else:
            #print(score)
            score_lst.append(score)
            current_district = row['district']
            score = 0
            score += row['score_2012']
    return score_lst

score_lst = district_score_2012()

#score_lst.append(100)

score_lst.append(691)

dfdistrictlevelmerged['scores_2012'] = score_lst

In [22]:
def district_score_2016():
    score_lst = []
    score = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            score += row['score_2016']
        else:
            #print(score)
            score_lst.append(score)
            current_district = row['district']
            score = 0
            score += row['score_2016']
    return score_lst

score_lst = district_score_2016()

#score_lst.append(100)

score_lst.append(691)

dfdistrictlevelmerged['scores_2016'] = score_lst

In [23]:
dfdistrictlevelmerged

,district,school count,scores_2012,scores_2016
0,Abby Kelley Foster Charter Public (District),1,1426,2840
1,Abington,3,2003,1803
2,Academy Of the Pacific Rim Charter Public (Dis...,1,493,1050
3,Acton-Boxborough,2,3867,2795
4,Acushnet,2,1466,1968
5,Adams-Cheshire,3,3516,3772
6,Advanced Math and Science Academy Charter (Dis...,1,969,1962
7,Agawam,7,6357,7608
8,Amesbury,4,3089,4498
9,Amherst,3,2452,2340


In [ ]:
def number_of_levels_2012():
    level_lst = []
    count = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            count += row['level_2012']
        else:
            current_district = row['district']
            level_lst.append(count)
            count = 0
            count += row['level_2012']
    return level_lst

level_lst_2012 = number_of_levels_2012()

level_lst_2012.append(2)  

print(len(level_lst_2012))

dfdistrictlevelmerged['levels 2012'] = level_lst_2012

In [ ]:
def number_of_levels_2016():
    level_lst = []
    count = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            count += row['level_2016']
        else:
            current_district = row['district']
            level_lst.append(count)
            count = 0
            count += row['level_2016']
    return level_lst

level_lst_2016 = number_of_levels_2016()

level_lst_2016.append(2)  

print(len(level_lst_2016))

dfdistrictlevelmerged['levels 2016'] = level_lst_2016

In [ ]:
def number_of_students_2012():
    student_lst = []
    count = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            count += int(row['student_count_2012'])
        else:
            current_district = row['district']
            student_lst.append(count)
            count = 0
            count += int(row['student_count_2012'])
    return student_lst

students_2012_lst = number_of_students_2012()

students_2012_lst.append(2)  

print(len(students_2012_lst))

dfdistrictlevelmerged['students 2012'] = students_2012_lst

In [ ]:
def number_of_students_2016():
    student_lst = []
    count = 0
    current_district = dfmerged.iloc[0]['district']
    for index, row in dfmerged.iterrows():
        if row['district'] == current_district:
            count += int(row['student_count_2016'])
        else:
            current_district = row['district']
            student_lst.append(count)
            count = 0
            count += int(row['student_count_2016'])
    return student_lst

students_2016_lst = number_of_students_2016()

students_2016_lst.append(2)  

print(len(students_2016_lst))

dfdistrictlevelmerged['students 2016'] = students_2016_lst

In [ ]:
dfdistrictlevelmerged[:4]

In [ ]:
# get number of schools at each level

def school_level_counts_2012():
    school_count_1 = []
    school_count_2 = []
    school_count_3 = []
    school_count_4 = []
    school_count_5 = []
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    
    current_district = df.iloc[0]['district']
    for index, row in df.iterrows():
        if row['district'] == current_district:
            
            if row['level_2012'] == 1:
                count1 += 1
            elif row['level_2012'] == 2:
                count2 += 1
            elif row['level_2012'] == 3:
                count3 += 1
            elif row['level_2012'] == 4:
                count4 += 1
            elif row['level_2012'] == 5:
                count5 += 1
            

        else:
            if index == 0:
                if row['level_2012'] == 1:
                    count1 += 1
                elif row['level_2012'] == 2:
                    count2 += 1
                elif row['level_2012'] == 3:
                    count3 += 1
                elif row['level_2012'] == 4:
                    count4 += 1
                elif row['level_2012'] == 5:
                    count5 += 1
                current_district = row['district']
            else:
                school_count_1.append(count1)
                school_count_2.append(count2)
                school_count_3.append(count3)
                school_count_4.append(count4)
                school_count_5.append(count5)
                count1 = 0
                count2 = 0
                count3 = 0
                count4 = 0
                count5 = 0
                if row['level_2012'] == 1:
                    count1 += 1
                elif row['level_2012'] == 2:
                    count2 += 1
                elif row['level_2012'] == 3:
                    count3 += 1
                elif row['level_2012'] == 4:
                    count4 += 1
                elif row['level_2012'] == 5:
                    count5 += 1
                current_district = row['district']
    school_count_1.append(0)
    school_count_2.append(2)
    school_count_3.append(0)
    school_count_4.append(0)
    school_count_5.append(0)
    return school_count_1, school_count_2, school_count_3, school_count_4, school_count_5 

level_1_schools_2012, level_2_schools_2012, level_3_schools_2012, level_4_schools_2012, \
level_5_schools_2012 = school_level_counts_2012()

print(len(level_2_schools_2012))

#schools_above_3_2012_lst.pop(0)

#schools_above_3_2012_lst.append(0)

dfdistrictlevel['level 1 2012'] = level_1_schools_2012
dfdistrictlevel['level 2 2012'] = level_2_schools_2012
dfdistrictlevel['level 3 2012'] = level_3_schools_2012
dfdistrictlevel['level 4 2012'] = level_4_schools_2012
dfdistrictlevel['level 5 2012'] = level_5_schools_2012

In [ ]:
# get number of schools level 3 or below at 2012 and at 2016

def school_level_counts_2016():
    school_count_1 = []
    school_count_2 = []
    school_count_3 = []
    school_count_4 = []
    school_count_5 = []
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    
    current_district = df.iloc[0]['district']
    for index, row in df.iterrows():
        if row['district'] == current_district:
            
            if row['level_2016'] == 1:
                count1 += 1
            elif row['level_2016'] == 2:
                count2 += 1
            elif row['level_2016'] == 3:
                count3 += 1
            elif row['level_2016'] == 4:
                count4 += 1
            elif row['level_2016'] == 5:
                count5 += 1
            

        else:
            if index == 0:
                if row['level_2016'] == 1:
                    count1 += 1
                elif row['level_2016'] == 2:
                    count2 += 1
                elif row['level_2016'] == 3:
                    count3 += 1
                elif row['level_2016'] == 4:
                    count4 += 1
                elif row['level_2016'] == 5:
                    count5 += 1
                current_district = row['district']
            else:
                school_count_1.append(count1)
                school_count_2.append(count2)
                school_count_3.append(count3)
                school_count_4.append(count4)
                school_count_5.append(count5)
                count1 = 0
                count2 = 0
                count3 = 0
                count4 = 0
                count5 = 0
                if row['level_2016'] == 1:
                    count1 += 1
                elif row['level_2016'] == 2:
                    count2 += 1
                elif row['level_2016'] == 3:
                    count3 += 1
                elif row['level_2016'] == 4:
                    count4 += 1
                elif row['level_2016'] == 5:
                    count5 += 1
                current_district = row['district']
    school_count_1.append(1)
    school_count_2.append(1)
    school_count_3.append(0)
    school_count_4.append(0)
    school_count_5.append(0)
    return school_count_1, school_count_2, school_count_3, school_count_4, school_count_5 

level_1_schools_2016, level_2_schools_2016, level_3_schools_2016, level_4_schools_2016, \
level_5_schools_2016 = school_level_counts_2016()

print(len(level_2_schools_2016))

#schools_above_3_2016_lst.pop(0)

#schools_above_3_2016_lst.append(0)

dfdistrictlevel['level 1 2016'] = level_1_schools_2016
dfdistrictlevel['level 2 2016'] = level_2_schools_2016
dfdistrictlevel['level 3 2016'] = level_3_schools_2016
dfdistrictlevel['level 4 2016'] = level_4_schools_2016
dfdistrictlevel['level 5 2016'] = level_5_schools_2016

In [ ]:
# gets the delta difference in school level between 2016 and 2012

deltalst = []
for index, row in dfdistrictlevel.iterrows():
    deltalst.append( row['level 3 2012'] + row['level 4 2012'] + row['level 5 2012']- (row['level 3 2016'] + row['level 4 2016'] + row['level 5 2016']) )
    
dfdistrictlevel['change in level'] = deltalst

In [ ]:
writer = pd.ExcelWriter('dflevels.xlsx')
dfdistrictlevel.to_excel(writer,'Sheet1')
writer.save()

In [ ]:
# new dataframe that uses percentage and removes districts with 4 or less schools

dfleveldelta = dfdistrictlevel[dfdistrictlevel['school count'] > 12]

dfleveldelta = dfleveldelta[dfleveldelta['change in level'] != 0]

#dfleveldelta['% schools higher than 3'] = dfleveldelta['% schools higher than 3'].apply(lambda x: round(x)) 

writer = pd.ExcelWriter('dflevels_schools_size_12.xlsx')
dfleveldelta.to_excel(writer,'Sheet1')
writer.save()

del dfleveldelta['change in level']
dfleveldelta

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfplot = dfleveldelta.sort_values(['school count'] , ascending=False)

N = len(dfplot)

width = 0.35       # the width of the bars


x1 = []
for index, row in dfplot.iterrows():
    x1.append( (row['level 3 2012'] + row['level 4 2012'] + row['level 5 2012']) )

x2 = []
for index, row in dfplot.iterrows():
    x2.append( (row['level 3 2016'] + row['level 4 2016'] + row['level 5 2016']) )

ind = np.arange(N)
fig, ax = plt.subplots()

rects1 = ax.bar(ind, x1, width, color='r', alpha=0.4)

rects2 = ax.bar(ind + width, x2, width, color='b', alpha=0.4)

# add some text for labels, title and axes ticks
ax.set_ylabel('number of schools level 3 or higher')
ax.set_xlabel('district')
ax.set_title('Net Number of Schools Level 3 or Higher 2012 vs. 2016')
ax.set_xticks(ind + width / len(dfplot) )
ax.set_xticklabels(dfplot['district'])

ax.legend((rects1[0], rects2[0]), ('2012', '2016'))

plt.setp(ax.get_xticklabels(), rotation=35, horizontalalignment='right')
        

plt.ylim([0,60])


plt.savefig('schools_level_3or_higher_2012vs2016.png', dpi=600, bbox_inches='tight')


plt.show()

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfplot = dfleveldelta.sort_values(['school count'] , ascending=False)

N = len(dfplot)

width = 0.35       # the width of the bars


x1 = []
for index, row in dfplot.iterrows():
    x1.append( (row['level 3 2012'] + row['level 4 2012'] + row['level 5 2012']) / row['school count'] )

x2 = []
for index, row in dfplot.iterrows():
    x2.append( (row['level 3 2016'] + row['level 4 2016'] + row['level 5 2016']) / row['school count'] )

ind = np.arange(N)
fig, ax = plt.subplots()

rects1 = ax.bar(ind, x1, width, color='blue', alpha=0.3)
rects1[6].set_color('red')

rects2 = ax.bar(ind + width, x2, width, color='lightgreen', alpha=0.4)
rects2[6].set_color('cyan')

# add some text for labels, title and axes ticks
ax.set_ylabel('percentage of level 3 or higher')
ax.set_xlabel('district')
ax.set_title('Percentage of Schools Level 3 or Higher out of Total Schools 2012 vs. 2016')
ax.set_xticks(ind + width -1 / len(dfplot) )
ax.set_xticklabels(dfplot['district'])

ax.legend((rects1[0], rects2[0]), ('2012', '2016'))

plt.setp(ax.get_xticklabels(), rotation=35, horizontalalignment='right')
        

plt.ylim([0,1])


#plt.savefig('percentage_schools_level_3or_higher_out_of_total_2012vs2016.png', dpi=600, bbox_inches='tight')


plt.show()

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfplot = dfleveldelta.sort_values(['school count'], ascending=False)

N = len(dfplot)

width = 0.35       # the width of the bars


x1 = []
for index, row in dfplot.iterrows():
    x1.append( (row['level 3 2012'])  )

x2 = []
for index, row in dfplot.iterrows():
    x2.append( (row['level 3 2016'])  )

ind = np.arange(N)
fig, ax = plt.subplots()

rects1 = ax.bar(ind, x1, width, color='blue', alpha=0.3)
rects1[6].set_color('red')

rects2 = ax.bar(ind + width, x2, width, color='lightgreen', alpha=0.4)
rects2[6].set_color('cyan')

# add some text for labels, title and axes ticks
ax.set_ylabel('number of level 3 schools')
ax.set_xlabel('district')
ax.set_title('number of level 3 schools 2012 vs. 2016')
ax.set_xticks(ind + width / len(dfplot) )
ax.set_xticklabels(dfplot['district'])

ax.legend((rects1[0], rects2[0]), ('2012', '2016'))

plt.setp(ax.get_xticklabels(), rotation=35, horizontalalignment='right')
        

plt.ylim([0,50])


#plt.savefig('schools_level_3_2012vs2016.png', dpi=600, bbox_inches='tight')


plt.show()

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfplot = dfleveldelta.sort_values(['school count'], ascending=False)

N = len(dfplot)

width = 0.35       # the width of the bars


x1 = []
for index, row in dfplot.iterrows():
    x1.append( (row['level 4 2012']) + (row['level 5 2012'])  )

x2 = []
for index, row in dfplot.iterrows():
    x2.append( (row['level 4 2016']) + (row['level 5 2016'])  )

ind = np.arange(N)
fig, ax = plt.subplots()

rects1 = ax.bar(ind, x1, width, color='blue', alpha=0.3)
rects1[6].set_color('red')

rects2 = ax.bar(ind + width, x2, width, color='lightgreen', alpha=0.4)
rects2[6].set_color('cyan')

# add some text for labels, title and axes ticks
ax.set_ylabel('number of level 4 and 5 schools')
ax.set_xlabel('district')
ax.set_title('Number of Level 4 and 5 Schools 2012 vs. 2016')
ax.set_xticks(ind + width / len(dfplot) )
ax.set_xticklabels(dfplot['district'])

ax.legend((rects1[0], rects2[0]), ('2012', '2016'))

plt.setp(ax.get_xticklabels(), rotation=35, horizontalalignment='right')
        

plt.ylim([0,13])


plt.savefig('schools_level_4-5_2012vs2016.png', dpi=600, bbox_inches='tight')


plt.show()

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

dfplot = dfleveldelta.sort_values(['school count'], ascending=False)

N = len(dfplot)

x = []
for index, row in dfplot.iterrows():
    #x.append(  (( row['level 3 2012'] + row['level 4 2012'] + row['level 5 2012']) -  (row['level 3 2016'] + row['level 4 2016'] + row['level 5 2016'])) / row['school count'] )
    x.append(  1 - ((row['level 3 2016'] + row['level 4 2016'] + row['level 5 2016']) /  ( row['level 3 2012'] + row['level 4 2012'] + row['level 5 2012']) ))
    

width = 0.95       # the width of the bars

ind = np.arange(N)
fig, ax = plt.subplots()

rects1 = ax.bar(ind, x, width, color='blue', alpha=0.3)
rects1[6].set_color('red')


# add some text for labels, title and axes ticks
ax.set_ylabel('Percentage of Schools')
ax.set_xlabel('district')
ax.set_title('Percentage of Schools that were level 3 or higher in 2012 and are level 2 or lower in 2016')
ax.set_xticks(ind + width / len(dfplot) )
ax.set_xticklabels(dfplot['district'])
plt.setp(ax.get_xticklabels(), rotation=35, horizontalalignment='right')

plt.ylim([0,.55])


plt.savefig('percent_schools_rose_from_lvl3_barchart.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt


x = dfplot['number of schools at level 3 or higher 2012']
y = dfplot['number of schools at level 3 or higher 2016']

colors = np.random.rand(20)

plt.scatter(x, y, s=(dfplot['% schools higher than 3'].apply(lambda x: x * 1000)), c='maroon', alpha=0.5)

plt.show()